Upload data to huggingface dataset

In [2]:
import pandas as pd
import os
import json
import os
from utils import split_data, convert_to_parquet, combine_window

In [8]:
file_path = "/home/ubuntu/multimodal/Dataset/Gas-raw/gas_text_time_final.csv"
df = pd.read_csv(file_path)


# for west coast only
# df['price'] = df.apply(lambda row: {
#     "West_Coast": row["West_Coast"]
# }, axis=1)
# for 8 regions
df['price'] = df.apply(lambda row: {
    "OT": row["OT"],
    "East_Coast": row["East_Coast"],
    "New_England": row["New_England"],
    "Central_Atlantic": row["Central_Atlantic"],
    "Lower_Atlantic": row["Lower_Atlantic"],
    "Midwest": row["Midwest"],
    "Gulf_Coast": row["Gulf_Coast"],
    "Rocky_Mountain": row["Rocky_Mountain"],
    "West_Coast": row["West_Coast"]
}, axis=1)
df['summary'] = df['formatted_text']

df['time_period'] = df.apply(lambda x: x['start_date'] + " - " + x['end_date'], axis=1)
df['summary_price'] = df.apply(lambda x: json.dumps({"time-period": x['time_period'], "summary": x['summary'], "gas_price": x['price']}), axis=1)
df['summary'] = df.apply(lambda x: json.dumps({"time-period": x['time_period'], "summary": x['summary']}), axis=1)

df['fut_summary_price'] = df['summary_price'].shift(-1)
df['fut_summary'] = df['summary'].shift(-1)

df = df[['summary_price', 'fut_summary_price', 'summary', 'fut_summary']]

df.to_csv('/home/ubuntu/multimodal/Dataset/Gas/all_regions.csv', index=False)

In [2]:
file_path = "/home/ubuntu/multimodal/Dataset/Gas/all_regions.csv"

split_data(file_path)  # Adjust the directory path as needed

# Case 3 text + number  => text + number

In [6]:
dir = '../Dataset/Gas'
window_sizes = [1, 2, 3]

def create_mixed_mixed(filename, window_size, unit):
    if filename.startswith('train') or filename.startswith('test') or filename.startswith('val'):
        path = os.path.join(dir, filename)
        summaries = pd.read_csv(path)

        if window_size == 1:
            window = f"{window_size} {unit}"
        else:
            window = f"{window_size} {unit}s"
        example_output = {}
        example_output[f"{unit}_{1+window_size}"] = {"time-period": "...", "summary": "...", "gas_price": "..."}
        example_output = json.dumps(example_output)
        instruction = f"Given the summary and the gas price for {window}, please predict the next 1 week's gas price and summary with in a JSON. The example output is {example_output}"

        df = summaries[['summary_price', 'fut_summary_price']].rename(columns={'summary_price': 'input', 'fut_summary_price': 'output'})
        df['instruction'] = instruction
        df['pred_output'] = "Not available"
        # skip the first and last historical_size days
        return combine_window(df, window_size, "week")

for window_size in window_sizes:
    dataframe_train = []
    dataframe_test = []
    dataframe_val = []
    for filename in os.listdir(dir):
        df = create_mixed_mixed(filename, window_size, "week")
        if filename.startswith('test'):
            dataframe_test.append(df)
        elif filename.startswith('train'):
            dataframe_train.append(df)
        elif filename.startswith('val'):
            dataframe_val.append(df)
        
    dataset_dict = convert_to_parquet(dataframe_test, dataframe_train, dataframe_val)
    token = os.getenv("HF_TOKEN")
    # Push the dataset to the Hugging Face Hub
    dataset_dict.push_to_hub(f"Howard881010/gas-{window_size}_week-mixed-mixed-west", token=token)

    # Load the dataset from Hugging Face Hub
    # load_from_huggingface("climate", f"Howard881010/gas-{window_size}_week-mixed-mixed", "mixed-mixed", f"{window_size}_week")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/601 [00:00<?, ?B/s]

# Case 4 text => text


In [3]:
dir = '../Dataset/Gas'
window_sizes = [1, 2, 3]

def create_text_text(filename, window_size, unit):
    if filename.startswith('train') or filename.startswith('test') or filename.startswith('val'):
        path = os.path.join(dir, filename)
        summaries = pd.read_csv(path)

        if window_size == 1:
            window = f"{window_size} {unit}"
        else:
            window = f"{window_size} {unit}s"
        example_output = {}
        example_output[f"{unit}_{1+window_size}"] = {"time-period": "...", "summary": "..."}
        example_output = json.dumps(example_output)
        instruction = f"Given the summary for {window}, please predict the next 1 week's summary with in a JSON. The example output is {example_output}"

        df = summaries[['summary', 'fut_summary']].rename(columns={'summary': 'input', 'fut_summary': 'output'})
        df['instruction'] = instruction
        df['pred_output'] = "Not available"
        # skip the first and last historical_size days
        return combine_window(df, window_size, "week")

for window_size in window_sizes:
    dataframe_train = []
    dataframe_test = []
    dataframe_val = []
    for filename in os.listdir(dir):
        df = create_text_text(filename, window_size, "week")
        if filename.startswith('test'):
            dataframe_test.append(df)
        elif filename.startswith('train'):
            dataframe_train.append(df)
        elif filename.startswith('val'):
            dataframe_val.append(df)
        
    dataset_dict = convert_to_parquet(dataframe_test, dataframe_train, dataframe_val)
    token = os.getenv("HF_TOKEN")
    # Push the dataset to the Hugging Face Hub
    dataset_dict.push_to_hub(f"Howard881010/gas-{window_size}_week-text-text-west", token=token)

    # Load the dataset from Hugging Face Hub
    # load_from_huggingface("climate", f"Howard881010/gas-{window_size}_week-text-text", "text-text", f"{window_size}_week")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/601 [00:00<?, ?B/s]

In [7]:
dir = '../Dataset/Gas'
dataframe_train = []
dataframe_test = []
dataframe_val = []
window_size = 3


for filename in os.listdir(dir):
    for size in range(1, window_size+1):
        df1 = create_text_text(filename, size, "week")
        df2 = create_mixed_mixed(filename, size, "week")
        if filename.startswith('test'):
            dataframe_test.append(df1)
            dataframe_test.append(df2)
        elif filename.startswith('train'):
            dataframe_train.append(df1)
            dataframe_train.append(df2)
        elif filename.startswith('val'):
            dataframe_val.append(df1)
            dataframe_val.append(df2)
        
dataset_dict = convert_to_parquet(dataframe_test, dataframe_train, dataframe_val)
token = os.getenv("HF_TOKEN")
# Push the dataset to the Hugging Face Hub
dataset_dict.push_to_hub(f"Howard881010/gas-west", token=token)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/651 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Howard881010/gas-west/commit/f2900be78f8d881b1771ce2cb95b62fa229b62f9', commit_message='Upload dataset', commit_description='', oid='f2900be78f8d881b1771ce2cb95b62fa229b62f9', pr_url=None, pr_revision=None, pr_num=None)